In [ ]:
#%pip install openai 

In [2]:
# 루트경로에 .env 파일을 만들고, OPENAI_API_KEY='{API_KEY}',  HUGGINGFACEHUB_API_TOKEN='{API_TOKEN}'식으로 각각 입력한다.
# 
# API 키를 환경변수로 관리하기 위한 .env설정 파일 로딩
import os
from dotenv import load_dotenv

load_dotenv() # API 키 정보 로드
print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")

KeyError: 'OPENAI_API_KEY'

In [1]:
%pip install requests pillow


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 사진의 의미를 자세히 설명해주세요. 한국어로 설명해주세요. 최대 5문장으로 설명을 간결하게 해주세요.
query = 'Please explain in detail the meaning of the photo. Please explain in Korean. Please keep your description concise with a maximum of 5 sentences.'

In [ ]:
# 카카오톡 url 이미지 경로
url = "https://talk.kakaocdn.net/dna/blJllF/bl6S3GjcuQo/TEeZqMPzV9KHm8uO6JnOVh/i_62fee87e33df.jpeg?credential=zf3biCPbmWRjbqf40YGePFLewdou7TIK&expires=1812587879&signature=gQrVVjT3ADAQRf6CGrOnh1z5y6Y%3D"
#url = '''http://k.kakaocdn.net/dn/BW8Ng/btsA7UjIXma/Eo2vvDdqrlS1dsFLjFMwJk/2x1.jpg'''

In [ ]:
# %pip install requests pillow
# url이미지 다운로드 후 512*512 사이즈 이상이면 512*512 사이즈로 줄여서 파일 저장
import os
import time
import requests
from PIL import Image
from io import BytesIO
import uuid
import datetime

def download_and_save_image(url, max_size=(512, 512)):
    
    # 랜덤 파일 이름 생성 (downloads/{랜덤}_{날짜}.jpg)
    random_filename = f"{uuid.uuid4().hex}_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}.jpg"
    save_path = f"downloads/{random_filename}"
    # downloads 폴더 없으면 생성
    os.makedirs("downloads", exist_ok=True)
    
    # 이미지 다운로드
    response = requests.get(url)
    if response.status_code == 200:
        # 이미지 열기
        img = Image.open(BytesIO(response.content))

        # 이미지 크기 확인 및 조정
        if img.size[0] > max_size[0] or img.size[1] > max_size[1]:
            img.thumbnail(max_size)

        # JPG 형식으로 저장
        img.save(save_path, "JPEG")
        print(f"이미지가 성공적으로 저장되었습니다: {save_path}")
    else:
        print(f"이미지 다운로드 실패. 상태 코드: {response.status_code}")

    return save_path

# 사용 예시
#url = "https://talk.kakaocdn.net/dna/blJllF/bl6S3GjcuQo/TEeZqMPzV9KHm8uO6JnOVh/i_62fee87e33df.jpeg?credential=zf3biCPbmWRjbqf40YGePFLewdou7TIK&expires=1812587879&signature=gQrVVjT3ADAQRf6CGrOnh1z5y6Y%3D"

start_time = time.time()

saved_image_path = download_and_save_image(url)

# 소요된 시간을 계산합니다.
end_time = time.time()
elapsed_time = "{:.2f}".format(end_time - start_time)
print(f'time:{elapsed_time}')

print(f"저장된 이미지 경로: {saved_image_path}")

In [ ]:
# openai vision으로 이미지 의미를 얻어옴
# => 모델은 gpt-4o 이
# 출처 : https://platform.openai.com/docs/guides/vision
import time
import base64
import requests

start_time = time.time()

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = saved_image_path

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}"
}

payload = {
  "model": "gpt-4o",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": query
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

# 소요된 시간을 계산합니다.
end_time = time.time()
elapsed_time = "{:.2f}".format(end_time - start_time)
print(f'time:{elapsed_time}')

print(response.json())

In [ ]:
res = response.json()
content = res['choices'][0]['message']['content']
print(content)

In [11]:
# 현재 사용량 얻어오기 
import requests

# URL 설정
url = "http://192.168.0.61:3000/api/get_limit"

# GET 요청 보내기
response = requests.get(url)

# 응답 확인
if response.status_code == 200:
    print("요청이 성공적으로 전송되었습니다.")
    print("응답 내용:", response.json())
else:
    print("요청이 실패했습니다. 상태 코드:", response.status_code)
    print("응답 내용:", response.text)

요청이 성공적으로 전송되었습니다.
응답 내용: {'credits_left': 2450, 'period': 'month', 'monthly_limit': 2500, 'monthly_usage': 50}


In [ ]:
# sudo로 content 내용 연동해서 노래 생성
import time
import requests
import json

start_time = time.time()

prompt = content
print(f'*prompt: {prompt}')
# URL 설정
url = "http://192.168.0.55:3000/api/generate"

# 요청 데이터 설정
data={
  "prompt": prompt,
  "make_instrumental": False,
  "wait_audio": False
}

# 헤더 설정 (JSON 데이터를 보내므로 Content-Type을 application/json으로 설정)
headers = {
    "Content-Type": "application/json"
}

# POST 요청 보내기
response = requests.post(url, data=json.dumps(data), headers=headers)

# 소요된 시간을 계산합니다.
end_time = time.time()
elapsed_time = "{:.2f}".format(end_time - start_time)
print(f'time:{elapsed_time}')

# 응답 확인
if response.status_code == 200:
    print("요청이 성공적으로 전송되었습니다.")
    print("응답 내용:", response.json())
else:
    print("요청이 실패했습니다. 상태 코드:", response.status_code)
    print("응답 내용:", response.text)

In [ ]:
# response에서 id만 뽑아냄 => id는 2개 만들어짐.
suno_res = response.json()

ids:list = []
for suno in suno_res:
    id = suno['id']
    if id:
        ids.append(id)

print(ids)

In [ ]:
# 실제 id를 전송해서 mp4, mp3 파일 경로 얻어옴
import requests

# 요청할 URL
url = "http://192.168.0.55:3000/api/get"

video_urls:list = []
audio_urls:list = []

for id in ids:
    # 쿼리 파라미터
    params = {
        "ids": id
    }
    
    # GET 요청 보내기
    mp_response = requests.get(url, params=params)
    
    # 응답 코드 확인
    print(f"Status Code: {mp_response.status_code}")
    
    # 응답 내용 출력
    print("Response Content:")
    data = mp_response.json()
    print(data)
    print(f'*'*50)   
    video_urls.append(data[0]['video_url'])
    audio_urls.append(data[0]['audio_url'])

print("*video_urls:")
print(video_urls)
print("*audio_urls:")
print(audio_urls)


In [ ]:
#%pip install fastapi uvicorn jinja2

In [10]:
import requests
import json
import webbrowser
import tempfile
import os
import atexit  
import uuid

# 요청을 보낼 URL
url = "http://192.168.0.61:8000/set_playlist"

# POST 요청의 body에 들어갈 데이터
data = {
    "urls": [
        "https://cdn1.suno.ai/d4d9e9c1-c842-4714-afe9-d5b15ea8bdc4.mp3"
    ]
}

# 데이터를 JSON 문자열로 변환
json_data = json.dumps(data)

# headers 설정 (Content-Type을 application/json으로 설정)
headers = {
    "Content-Type": "application/json"
}

# POST 요청 보내기
response = requests.post(url, data=json_data, headers=headers)

# 응답 코드와 내용 출력
print(f"Status Code: {response.status_code}")
print(f"response: {response.text}")


Status Code: 200
response: <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>FastAPI Music Player</title>
    <script src="http://192.168.0.55:8000/static/player.js"></script>
</head>
<body>
    <h1>Music Player</h1>
    <audio id="audioPlayer" controls autoplay></audio>
    <script>
        const tracks = ["https://cdn1.suno.ai/d4d9e9c1-c842-4714-afe9-d5b15ea8bdc4.mp3"];
        setupPlayer(tracks);
    </script>
</body>
</html>


In [10]:
import requests
import json
import webbrowser
import tempfile
import os
import atexit  
import uuid

# 요청을 보낼 URL
url = "http://192.168.0.55:8000/set_playlist"

# POST 요청의 body에 들어갈 데이터
data = {
    "urls": [
        "https://cdn1.suno.ai/d4d9e9c1-c842-4714-afe9-d5b15ea8bdc4.mp3"
    ]
}

# 데이터를 JSON 문자열로 변환
json_data = json.dumps(data)

# headers 설정 (Content-Type을 application/json으로 설정)
headers = {
    "Content-Type": "application/json"
}

# POST 요청 보내기
response = requests.post(url, data=json_data, headers=headers)

# 응답 코드와 내용 출력
print(f"Status Code: {response.status_code}")
print(f"response: {response.text}")


Status Code: 200
response: <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>FastAPI Music Player</title>
    <script src="http://192.168.0.55:8000/static/player.js"></script>
</head>
<body>
    <h1>Music Player</h1>
    <audio id="audioPlayer" controls autoplay></audio>
    <script>
        const tracks = ["https://cdn1.suno.ai/d4d9e9c1-c842-4714-afe9-d5b15ea8bdc4.mp3"];
        setupPlayer(tracks);
    </script>
</body>
</html>


In [3]:
import os
import webbrowser
temp_path = f"10916f468be743bfbf3bf7bd7a812c2d.html"
abs_path = f'd:\\docker\\jupyter\\jupyterdir\\tmp\\{temp_path}'


print(abs_path)
webbrowser.open(f"file://{abs_path}")

d:\docker\jupyter\jupyterdir\tmp\10916f468be743bfbf3bf7bd7a812c2d.html


False